In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from transformers import AutoModel, AutoTokenizer, get_linear_schedule_with_warmup
from torch.optim import AdamW
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
import json
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

!pip install transformers datasets torch underthesea scikit-learn -q

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
TEST_MODE = False
print(f"!!! 🚀 CHẾ ĐỘ TEST: {TEST_MODE} 🚀 !!!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.6 MB/s eta 0:00:00
Using device: cuda
!!! 🚀 CHẾ ĐỘ TEST: False 🚀 !!!


In [ ]:
# ==========================================================
# 1️⃣ LOAD & TIỀN XỬ LÝ DỮ LIỆU
# ==========================================================
data_path = '/content/drive/Shareddrives/Deeplearning/data/spam_emails_new(4).csv'
df = pd.read_csv(data_path)

# Xử lý cột trống
df["subject"] = df["subject"].fillna("[No Subject]")
df["body"] = df["body"].fillna("")
df["text"] = (df["subject"] + " " + df["body"]).str.strip()

# Xử lý loại bỏ ký tự đặc biệt và khoảng trống thừa
import re

def clean_text(text):
    if isinstance(text, str):
        # Loại bỏ các ký tự đặc biệt, chỉ giữ lại chữ cái, số, và dấu câu cơ bản
        text = re.sub(r'[^\w\s.,!?;:()\-]', '', text)
        # Thay thế nhiều khoảng trắng liên tiếp bằng một khoảng trắng
        text = re.sub(r'\s+', ' ', text)
        # Loại bỏ khoảng trắng ở đầu và cuối
        text = text.strip()
    return text

# Áp dụng hàm clean_text cho các cột
df["subject"] = df["subject"].apply(clean_text)
df["body"] = df["body"].apply(clean_text)
df["text"] = df["text"].apply(clean_text)

# --- KIỂM TRA VÀ XỬ LÝ DỮ LIỆU TRƯỚC KHI MAPPING ---
print("Kiểm tra dữ liệu trước khi mapping:")
print("Giá trị unique trong 'type':", df["type"].unique())
print("Giá trị unique trong 'priority':", df["priority"].unique())
print("Số lượng NaN trong 'type':", df["type"].isna().sum())
print("Số lượng NaN trong 'priority':", df["priority"].isna().sum())

# Xử lý giá trị 'type' (có thể là lỗi dữ liệu)
df = df[df["type"] != "type"]  # Loại bỏ hàng có giá trị 'type'

# Xử lý các giá trị NaN
df.dropna(subset=["type", "priority"], inplace=True)

# --- Encode nhãn (CỐ ĐỊNH THỨ TỰ) ---
type_labels_sorted = sorted(df["type"].unique())
type2id = {label: i for i, label in enumerate(type_labels_sorted)}
id2type = {i: label for label, i in type2id.items()}

# FIX: Cố định thứ tự cho Priority
PRIORITY_ORDER = ['low', 'medium', 'high']
priority2id = {label: i for i, label in enumerate(PRIORITY_ORDER)}
id2priority = {i: label for i, label in enumerate(PRIORITY_ORDER)}

df["type_id"] = df["type"].map(type2id)
df["priority_id"] = df["priority"].map(priority2id)

# --- KIỂM TRA SAU KHI MAPPING ---
print("\nKiểm tra sau khi mapping:")
print("Số lượng NaN trong type_id:", df["type_id"].isna().sum())
print("Số lượng NaN trong priority_id:", df["priority_id"].isna().sum())
print("Type Mapping:", type2id)
print("Priority Mapping:", priority2id)

# Loại bỏ các hàng có NaN trong type_id hoặc priority_id sau khi mapping
df.dropna(subset=["type_id", "priority_id"], inplace=True)

# Chuyển đổi sang int sau khi đảm bảo không còn NaN
df["type_id"] = df["type_id"].astype(int)
df["priority_id"] = df["priority_id"].astype(int)

print(f"\nKích thước dữ liệu sau khi xử lý: {df.shape}")

# --- Split data (Stratify trên cả 2 cột) ---
# Kiểm tra lại trước khi split
print("\nKiểm tra cuối cùng trước khi split:")
print("NaN trong type_id:", df["type_id"].isna().sum())
print("NaN trong priority_id:", df["priority_id"].isna().sum())

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42,
                                     stratify=df[["type_id", "priority_id"]])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42,
                                   stratify=temp_df[["type_id", "priority_id"]])

print(f"Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}")

if TEST_MODE:
    print("--- ĐANG GIẢM DỮ LIỆU CHO TEST MODE ---")
    train_df = train_df.sample(n=min(100, len(train_df)), random_state=42).reset_index(drop=True)
    val_df = val_df.sample(n=min(50, len(val_df)), random_state=42).reset_index(drop=True)
    test_df = test_df.sample(n=min(50, len(test_df)), random_state=42).reset_index(drop=True)
    print(f"Test Mode: Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}")

Kiểm tra dữ liệu trước khi mapping:
Giá trị unique trong 'type': ['Incident' 'Request' 'Problem' 'Change']
Giá trị unique trong 'priority': ['high' 'medium' 'low']
Số lượng NaN trong 'type': 0
Số lượng NaN trong 'priority': 0

Kiểm tra sau khi mapping:
Số lượng NaN trong type_id: 0
Số lượng NaN trong priority_id: 0
Type Mapping: {'Change': 0, 'Incident': 1, 'Problem': 2, 'Request': 3}
Priority Mapping: {'low': 0, 'medium': 1, 'high': 2}

Kích thước dữ liệu sau khi xử lý: (16251, 19)

Kiểm tra cuối cùng trước khi split:
NaN trong type_id: 0
NaN trong priority_id: 0
Train: (11375, 19), Val: (2438, 19), Test: (2438, 19)


In [ ]:
# ==========================================================
# 2️⃣ XỬ LÝ MẤT CÂN BẰNG (TÍNH TOÁN 1 LẦN)
# ==========================================================

# --- 1. Tính Class Weights cho PRIORITY (dùng trong Weighted Loss) ---
# Đảm bảo thứ tự đúng như mapping
pri_classes_idx = np.array([priority2id[label] for label in PRIORITY_ORDER])

priority_class_weights = compute_class_weight(
    class_weight="balanced",
    classes=pri_classes_idx,
    y=train_df["priority_id"]
)
priority_class_weights = torch.tensor(priority_class_weights, dtype=torch.float32).to(device)

print("Priority Task (Weighted Loss):")
for i, label in enumerate(PRIORITY_ORDER):
    print(f"  Class {i} ({label}): weight = {priority_class_weights[i]:.4f}")


Priority Task (Weighted Loss):
  Class 0 (low): weight = 1.0122
  Class 1 (medium): weight = 0.9680
  Class 2 (high): weight = 1.0215


In [ ]:
# ==========================================================
# 3️⃣ DATASET CLASS (Sửa __getitem__)
# ==========================================================
class EmailDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=384):
        self.texts = df["text"].tolist()
        self.types = df["type_id"].tolist()
        self.priorities = df["priority_id"].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # __getitem__ chỉ xử lý 1 index (int)
        text = self.texts[idx]
        type_label = self.types[idx]
        priority_label = self.priorities[idx]

        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0), # Squeeze để bỏ dim batch
            "attention_mask": enc["attention_mask"].squeeze(0),
            "type_label": torch.tensor(type_label, dtype=torch.long),
            "priority_label": torch.tensor(priority_label, dtype=torch.long),
        }

In [ ]:
# ==========================================================
# 4️⃣ MÔ HÌNH MULTI-TASK
# ==========================================================

# (Bạn có thể đặt class FocalLoss ở đây nếu muốn dùng)
# class FocalLoss(nn.Module): ...

class DistilBERTMultiTask(nn.Module):
    def __init__(self, model_name, num_type, num_priority, pri_weights):
        super().__init__()
        self.distilbert = AutoModel.from_pretrained(model_name)
        hidden = self.distilbert.config.hidden_size
        self.dropout = nn.Dropout(0.3)
        self.type_head = nn.Linear(hidden, num_type)
        self.priority_head = nn.Linear(hidden, num_priority)

        # Khởi tạo loss functions ở đây
        self.type_loss_fn = nn.CrossEntropyLoss()
        self.priority_loss_fn = nn.CrossEntropyLoss(weight=pri_weights)

        # (Nếu muốn dùng FocalLoss, hãy thay self.priority_loss_fn bằng:
        # self.priority_loss_fn = FocalLoss(alpha=pri_weights)

    def forward(self, input_ids, attention_mask, type_label=None, priority_label=None):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = outputs.last_hidden_state[:, 0, :]
        pooled = self.dropout(pooled)

        type_logits = self.type_head(pooled)
        priority_logits = self.priority_head(pooled)

        loss = None
        if type_label is not None and priority_label is not None:
            type_loss = self.type_loss_fn(type_logits, type_label)
            pri_loss = self.priority_loss_fn(priority_logits, priority_label)

            # TỐI ƯU: Đảo trọng số, ưu tiên task khó (priority)
            loss = (0.3 * type_loss) + (0.7 * pri_loss)

        return {
            "loss": loss,
            "type_logits": type_logits,
            "priority_logits": priority_logits
        }

In [ ]:
# ==========================================================
# 5️⃣ SETUP MODEL, LOADER
# ==========================================================
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Truyền class weights vào model
model = DistilBERTMultiTask(
    model_name,
    len(type2id),
    len(priority2id),
    pri_weights=priority_class_weights # Truyền trọng số đã tính
).to(device)

# Tạo datasets
train_dataset = EmailDataset(train_df, tokenizer)
val_dataset = EmailDataset(val_df, tokenizer)
test_dataset = EmailDataset(test_df, tokenizer)

# Tạo datasets
train_dataset = EmailDataset(train_df, tokenizer)
val_dataset = EmailDataset(val_df, tokenizer)
test_dataset = EmailDataset(test_df, tokenizer)

# --- THÊM LOGIC SAMPLER VÀO ĐÂY (VÌ GIỜ MỚI CÓ train_df CUỐI CÙNG) ---
# Tính toán lại dựa trên train_df (đã được sample nếu TEST_MODE=True)
type_classes_idx = np.array([type2id[label] for label in type_labels_sorted])
type_class_weights_array = compute_class_weight(
    class_weight="balanced",
    classes=type_classes_idx,
    y=train_df["type_id"]
)
sample_weights = [type_class_weights_array[label] for label in train_df["type_id"]]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
print(f"Đã tạo sampler cho {len(sample_weights)} sample.")
# --- KẾT THÚC LOGIC SAMPLER ---

# Tạo DataLoaders
BATCH_SIZE = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=sampler # Sử dụng sampler vừa tính
)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Optimizer & Scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
NUM_EPOCHS = 15
total_steps = len(train_loader) * NUM_EPOCHS
warmup_steps = int(total_steps * 0.1) # 10% warmup

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Đã tạo sampler cho 11375 sample.


In [ ]:
# ==========================================================
# 6️⃣ HÀM EVALUATE (Gọn gàng)
# ==========================================================

def evaluate(model, loader, device):
    model.eval()
    preds_type, labels_type = [], []
    preds_priority, labels_priority = [], []
    total_val_loss = 0

    with torch.no_grad():
        for batch in loader:
            # Gửi mọi thứ lên device
            inputs = {k: v.to(device) for k, v in batch.items()}

            # Truyền cả label để model tính loss
            outputs = model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                type_label=inputs["type_label"],
                priority_label=inputs["priority_label"]
            )

            # Loss này đã được tính với trọng số (0.3/0.7)
            total_val_loss += outputs["loss"].item()

            # Lấy predictions
            pred_type = torch.argmax(outputs["type_logits"], dim=1)
            pred_priority = torch.argmax(outputs["priority_logits"], dim=1)

            preds_type.extend(pred_type.cpu().numpy())
            labels_type.extend(inputs["type_label"].cpu().numpy())
            preds_priority.extend(pred_priority.cpu().numpy())
            labels_priority.extend(inputs["priority_label"].cpu().numpy())

    avg_loss = total_val_loss / len(loader)

    # In report
    print("\n--- Validation Report ---")
    print("📌 Task 1: TYPE classification:")
    print(classification_report(labels_type, preds_type, target_names=type_labels_sorted, zero_division=0))
    print("\n📌 Task 2: PRIORITY classification:")
    print(classification_report(labels_priority, preds_priority, target_names=PRIORITY_ORDER, zero_division=0))

    # Tính toán metrics để trả về
    metrics = {
        'val_loss': avg_loss,
        'type_acc': accuracy_score(labels_type, preds_type),
        'type_f1': f1_score(labels_type, preds_type, average='weighted'),
        'pri_acc': accuracy_score(labels_priority, preds_priority),
        'pri_f1': f1_score(labels_priority, preds_priority, average='weighted')
    }
    # Thêm F1 trung bình để dễ theo dõi
    metrics['avg_f1'] = (metrics['type_f1'] + metrics['pri_f1']) / 2

    return metrics

In [ ]:
# ==========================================================
# 7️⃣ TRAINING LOOP (Tối ưu với AMP)
# ==========================================================
import os
from torch.cuda.amp import GradScaler, autocast # <-- THÊM IMPORT NÀY

os.makedirs("./multitask_checkpoints", exist_ok=True)

# Cài đặt cho training thật (nếu tắt TEST_MODE)
if TEST_MODE:
    NUM_EPOCHS = 2
    PATIENCE = 2
else:
    NUM_EPOCHS = 15 # Train 10 epoch
    PATIENCE = 5    # Patience 3 cho training thật

patience_counter = 0
best_metric = 0.0
best_model_path = "./multitask_checkpoints/best_model_by_f1.pt"

# KHỞI TẠO SCALER (cho AMP)
scaler = GradScaler()

print("--- BẮT ĐẦU TRAINING ---")
print(f"Số Epochs: {NUM_EPOCHS} | Patience: {PATIENCE}")


for epoch in range(NUM_EPOCHS):
    # --- TRAINING ---
    model.train()
    total_train_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]"):
        optimizer.zero_grad()

        inputs = {k: v.to(device) for k, v in batch.items()}

        # --- BỌC FORWARD PASS BẰNG AUTOCAST ---
        with autocast():
            outputs = model(**inputs) # Truyền tất cả
            loss = outputs["loss"]
        # ------------------------------------

        # --- SỬA CÁCH BACKWARD (dùng Scaler) ---
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update() # Cập nhật scaler
        # ------------------------------------

        scheduler.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f"\nEpoch {epoch+1} | Avg Train Loss: {avg_train_loss:.4f}")

    # --- VALIDATION ---
    # (Hàm evaluate không cần autocast vì nó đã có torch.no_grad())
    val_metrics = evaluate(model, val_loader, device)

    print(f"\nEpoch {epoch+1} [Validation Metrics]:")
    print(f"  Val Loss: {val_metrics['val_loss']:.4f} | Avg F1: {val_metrics['avg_f1']:.4f}")
    print(f"  Type (Acc/F1): {val_metrics['type_acc']:.4f} / {val_metrics['type_f1']:.4f}")
    print(f"  Pri (Acc/F1):  {val_metrics['pri_acc']:.4f} / {val_metrics['pri_f1']:.4f}")

    # --- CHECKPOINTING & EARLY STOPPING (DỰA TRÊN VALIDATION) ---
    current_metric = val_metrics['avg_f1']

    if current_metric > best_metric:
        best_metric = current_metric
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ New best model saved! (Avg F1: {best_metric:.4f})")
    else:
        patience_counter += 1
        print(f"Patience: {patience_counter}/{PATIENCE}")
        if patience_counter >= PATIENCE:
            print("🚫 Early stopping triggered!")
            break

# --- FINAL EVALUATION (trên TEST SET) ---
print("\n--- TRAINING HOÀN TẤT ---")
print(f"Loading best model from {best_model_path} for final evaluation.")

model.load_state_dict(torch.load(best_model_path))

print("\n===== 🏁 FINAL EVALUATION ON TEST SET 🏁 =====")
test_metrics = evaluate(model, test_loader, device)

print(f"\nFinal Test Metrics (from best model):")
print(f"  Avg F1: {test_metrics['avg_f1']:.4f}")
print(f"  Type F1: {test_metrics['type_f1']:.4f}")
print(f"  Pri F1: {test_metrics['pri_f1']:.4f}")

--- BẮT ĐẦU TRAINING ---
Số Epochs: 15 | Patience: 5


Epoch 1/15 [Train]: 100%|██████████| 356/356 [01:43<00:00,  3.43it/s]



Epoch 1 | Avg Train Loss: 1.1246

--- Validation Report ---
📌 Task 1: TYPE classification:
              precision    recall  f1-score   support

      Change       0.49      0.88      0.63       244
    Incident       0.64      0.75      0.69       968
     Problem       0.27      0.12      0.17       511
     Request       0.85      0.75      0.80       715

    accuracy                           0.63      2438
   macro avg       0.56      0.63      0.57      2438
weighted avg       0.61      0.63      0.61      2438


📌 Task 2: PRIORITY classification:
              precision    recall  f1-score   support

         low       0.95      0.46      0.62       803
      medium       0.41      0.21      0.28       838
        high       0.41      0.83      0.54       797

    accuracy                           0.49      2438
   macro avg       0.59      0.50      0.48      2438
weighted avg       0.59      0.49      0.48      2438


Epoch 1 [Validation Metrics]:
  Val Loss: 0.9343 | Avg 

Epoch 2/15 [Train]: 100%|██████████| 356/356 [01:42<00:00,  3.49it/s]



Epoch 2 | Avg Train Loss: 0.8208

--- Validation Report ---
📌 Task 1: TYPE classification:
              precision    recall  f1-score   support

      Change       0.90      0.98      0.94       244
    Incident       0.80      0.30      0.44       968
     Problem       0.39      0.84      0.54       511
     Request       0.98      0.97      0.97       715

    accuracy                           0.68      2438
   macro avg       0.77      0.77      0.72      2438
weighted avg       0.78      0.68      0.67      2438


📌 Task 2: PRIORITY classification:
              precision    recall  f1-score   support

         low       0.54      0.65      0.59       803
      medium       0.44      0.38      0.41       838
        high       0.49      0.44      0.46       797

    accuracy                           0.49      2438
   macro avg       0.49      0.49      0.49      2438
weighted avg       0.49      0.49      0.49      2438


Epoch 2 [Validation Metrics]:
  Val Loss: 0.7872 | Avg 

...


Epoch 13 | Avg Train Loss: 0.4340

--- Validation Report ---
📌 Task 1: TYPE classification:
              precision    recall  f1-score   support

      Change       0.95      1.00      0.97       244
    Incident       0.80      0.65      0.72       968
     Problem       0.51      0.68      0.58       511
     Request       0.99      0.98      0.99       715

    accuracy                           0.79      2438
   macro avg       0.81      0.83      0.82      2438
weighted avg       0.81      0.79      0.79      2438


📌 Task 2: PRIORITY classification:
              precision    recall  f1-score   support

         low       0.68      0.69      0.69       803
      medium       0.52      0.47      0.49       838
        high       0.59      0.64      0.61       797

    accuracy                           0.60      2438
   macro avg       0.60      0.60      0.60      2438
weighted avg       0.59      0.60      0.59      2438


Epoch 13 [Validation Metrics]:
  Val Loss: 0.7432 | Av

Epoch 14/15 [Train]: 100%|██████████| 356/356 [01:42<00:00,  3.48it/s]



Epoch 14 | Avg Train Loss: 0.4263

--- Validation Report ---
📌 Task 1: TYPE classification:
              precision    recall  f1-score   support

      Change       0.95      1.00      0.98       244
    Incident       0.81      0.65      0.72       968
     Problem       0.51      0.69      0.59       511
     Request       0.99      0.98      0.99       715

    accuracy                           0.79      2438
   macro avg       0.82      0.83      0.82      2438
weighted avg       0.81      0.79      0.80      2438


📌 Task 2: PRIORITY classification:
              precision    recall  f1-score   support

         low       0.72      0.68      0.70       803
      medium       0.53      0.52      0.52       838
        high       0.60      0.63      0.62       797

    accuracy                           0.61      2438
   macro avg       0.61      0.61      0.61      2438
weighted avg       0.61      0.61      0.61      2438


Epoch 14 [Validation Metrics]:
  Val Loss: 0.7471 | Av

Epoch 15/15 [Train]: 100%|██████████| 356/356 [01:44<00:00,  3.42it/s]



Epoch 15 | Avg Train Loss: 0.4227

--- Validation Report ---
📌 Task 1: TYPE classification:
              precision    recall  f1-score   support

      Change       0.95      1.00      0.98       244
    Incident       0.81      0.64      0.71       968
     Problem       0.51      0.71      0.59       511
     Request       0.99      0.98      0.99       715

    accuracy                           0.79      2438
   macro avg       0.82      0.83      0.82      2438
weighted avg       0.81      0.79      0.79      2438


📌 Task 2: PRIORITY classification:
              precision    recall  f1-score   support

         low       0.71      0.69      0.70       803
      medium       0.52      0.55      0.54       838
        high       0.62      0.59      0.61       797

    accuracy                           0.61      2438
   macro avg       0.62      0.61      0.61      2438
weighted avg       0.61      0.61      0.61      2438


Epoch 15 [Validation Metrics]:
  Val Loss: 0.7451 | Av

In [ ]:
# ==========================================================
# 8️⃣ SAO CHÉP MODEL SANG GOOGLE DRIVE
# ==========================================================
import shutil

# Đường dẫn đích trên Google Drive của bạn
# (Nó sẽ lưu vào thư mục gốc 'My Drive')
drive_save_path = "/content/drive/Shareddrives/Deeplearning/best_model.pt"

try:
    # Sao chép tệp model tốt nhất (đã được lưu ở Cell 7)
    # từ máy ảo Colab sang thư mục Google Drive của bạn
    shutil.copy(best_model_path, drive_save_path)
    print(f"✅ Sao chép thành công!")
    print(f"Model đã được lưu vĩnh viễn tại: {drive_save_path}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp model tại '{best_model_path}'.")
except Exception as e:
    print(f"Lỗi khi sao chép file: {e}")

✅ Sao chép thành công!
Model đã được lưu vĩnh viễn tại: /content/drive/Shareddrives/Deeplearning/best_model.pt
